In [1]:
import torch
torch.__version__

'1.6.0'

In [4]:
import os
import re

import cv2
import numpy as np
# import torch
from skimage import io
from torch import nn
from torchvision import models
import matplotlib.pyplot as plt

In [6]:
import os
import warnings

import mmcv
import torch
from mmcv import Config, DictAction
from mmcv.cnn import fuse_conv_bn
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import (get_dist_info, init_dist, load_checkpoint,
                         wrap_fp16_model)

from mmdet.apis import multi_gpu_test, single_gpu_test
from mmdet.datasets import (build_dataloader, build_dataset,
                            )
from mmdet.models import build_detector

In [8]:
cfg = Config.fromfile('/data2/guobo/01_SHIPRSDET/TrainTestv2/work_dirs/v1307_roitrans_r50_obbGRoI_noBN_noPre/v1307_roitrans_r50_obbGRoI_noBN_noPre.py')

In [9]:
model = build_detector(cfg.model, test_cfg=cfg.get('test_cfg'))

2022-04-21 10:09:44,454 - mmdet - INFO - load model from: torchvision://resnet50
2022-04-21 10:09:44,457 - mmdet - INFO - load checkpoint from torchvision path: torchvision://resnet50
2022-04-21 10:09:44,801 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias



In [12]:
checkpoint = load_checkpoint(model, "/data2/guobo/01_SHIPRSDET/TrainTestv2/work_dirs/v1307_roitrans_r50_obbGRoI_noBN_noPre/epoch_69.pth", map_location='cuda')

load checkpoint from local path: /data2/guobo/01_SHIPRSDET/TrainTestv2/work_dirs/v1307_roitrans_r50_obbGRoI_noBN_noPre/epoch_69.pth


In [13]:
model.CLASSES = checkpoint['meta']['CLASSES']

In [14]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
    dataset,
    samples_per_gpu=1,
    workers_per_gpu=cfg.data.workers_per_gpu,
    dist=False,
    shuffle=False)

In [16]:
data = []
for i, t in enumerate(data_loader):
    tmp = {}
    tmp['img'] = t['img']
    tmp['img_metas'] = t['img_metas'][0].data[0]
    data.append(tmp)

In [18]:
data[1]

{'img': [tensor([[[[-0.1314, -0.2684, -0.1999,  ..., -0.9534, -0.9534, -0.9534],
            [-0.0972, -0.1486, -0.0801,  ..., -0.9534, -0.9534, -0.9534],
            [-0.0116, -0.0116, -0.0116,  ..., -0.9534, -0.9534, -0.9534],
            ...,
            [ 0.0056,  0.0056,  0.0056,  ...,  0.0056,  0.0056,  0.0056],
            [ 0.0056,  0.0056,  0.0056,  ...,  0.0056,  0.0056,  0.0056],
            [ 0.0056,  0.0056,  0.0056,  ...,  0.0056,  0.0056,  0.0056]],
  
           [[ 0.2752,  0.1702,  0.2227,  ..., -0.5476, -0.5476, -0.5476],
            [ 0.3102,  0.2752,  0.3102,  ..., -0.5476, -0.5476, -0.5476],
            [ 0.3803,  0.3803,  0.3803,  ..., -0.5476, -0.5476, -0.5476],
            ...,
            [-0.0049, -0.0049, -0.0049,  ..., -0.0049, -0.0049, -0.0049],
            [-0.0049, -0.0049, -0.0049,  ..., -0.0049, -0.0049, -0.0049],
            [-0.0049, -0.0049, -0.0049,  ..., -0.0049, -0.0049, -0.0049]],
  
           [[ 0.4265,  0.3568,  0.3916,  ..., -0.2707, -0.2707,

In [19]:
# cfg.nms_pre = 1000
# cfg.min_bbox_size = 1
# cfg.nms_thr = 10
# cfg.nms_post = 1000
test_cfg = model.rpn_head.test_cfg

# img_metas = data['img_metas']
####################################################################################3
# 此處重寫了rpn_head.get_bboxes，因爲原函數使用detach把gradients的backprop刪了無法backward（）
#######################################################################################

def get_bboxes_tmp(rpn_outs, img_metas, test_cfg):
    
    with_nms = False
    rescale = False
    cls_scores = rpn_outs[0]
    bbox_preds = rpn_outs[1]

    num_levels = len(cls_scores)

    device = cls_scores[0].device
    featmap_sizes = [cls_scores[i].shape[-2:] for i in range(num_levels)]
    mlvl_anchors = model.rpn_head.anchor_generator.grid_anchors(
        featmap_sizes, device=device)

    result_list = []
    for img_id in range(len(img_metas)):
        cls_score_list = [
            cls_scores[i][img_id] for i in range(num_levels) # 此處不再使用detach()
        ]
        bbox_pred_list = [
            bbox_preds[i][img_id] for i in range(num_levels) # 此處不再使用detach()
        ]
        img_shape = img_metas[img_id]['img_shape']
        scale_factor = img_metas[img_id]['scale_factor']

        if with_nms:
            # some heads don't support with_nms argument
            proposals = model.rpn_head._get_bboxes_single(cls_score_list,
                                                bbox_pred_list,
                                                mlvl_anchors, img_shape,
                                                scale_factor, test_cfg, rescale)
        else:
            proposals = model.rpn_head._get_bboxes_single(cls_score_list,
                                                bbox_pred_list,
                                                mlvl_anchors, img_shape,
                                                scale_factor, test_cfg, rescale)
        result_list.append(proposals)
    
    return result_list

In [20]:
class GradCAM(object):
    """
    1: 网络不更新梯度,输入需要梯度更新
    2: 使用目标类别的得分做反向传播
    """

    def __init__(self, net, layer_name):
        self.net = net
        self.layer_name = layer_name
        self.feature = None
        self.gradient = None
        self.net.eval()
        self.handlers = []
        self._register_hook()

    def _get_features_hook(self, module, input, output):
        self.feature = output
        # print("feature shape:{}".format(output.size()))

    def _get_grads_hook(self, module, input_grad, output_grad):
        """

        :param input_grad: tuple, input_grad[0]: None
                                   input_grad[1]: weight
                                   input_grad[2]: bias
        :param output_grad:tuple,长度为1
        :return:
        """
        self.gradient = output_grad[0]
        # print('gradient:', self.gradient)

    def _register_hook(self):
        for (name, module) in self.net.named_modules():
            if name == self.layer_name:
                self.handlers.append(module.register_forward_hook(self._get_features_hook))
                self.handlers.append(module.register_backward_hook(self._get_grads_hook))
#         module = self.net.rpn.head.bbox_pred
#         self.handlers.append(module.register_forward_hook(self._get_features_hook))
#         self.handlers.append(module.register_backward_hook(self._get_grads_hook))
            
                # print(self.handlers)

    def remove_handlers(self):
        for handle in self.handlers:
            handle.remove()

    def __call__(self, inputs, index=0):
        """

        :param inputs: {"image": [C,H,W], "height": height, "width": width}
        :param index: 第几个边框
        :return:
        """
        self.net.zero_grad()
        # output = self.net([inputs])
        x = self.net.extract_feat(inputs['img'][0])
        rpn_outs = self.net.rpn_head(x)
        result_list = get_bboxes_tmp(rpn_outs,inputs['img_metas'],self.net.rpn_head.test_cfg)
        res= self.net.roi_head.simple_test_bboxes(
            x, inputs['img_metas'], result_list, self.net.roi_head.test_cfg, rescale=True)
        # print(output)
        score = res[0][0][index][4]
        # proposal_idx = output[0]['labels'][index]  # box来自第几个proposal
        # print(score)
        score.backward()
        # print('gradient:', self.gradient)

        # gradient = self.gradient[proposal_idx].cpu().data.numpy()  # [C,H,W]
        gradient = self.gradient.cpu().data.numpy().squeeze()
        
        weight = np.mean(gradient, axis=(1, 2))  # [C]

        # feature = self.feature[proposal_idx].cpu().data.numpy()  # [C,H,W]
        feature = self.feature.cpu().data.numpy().squeeze()

        cam = feature * weight[:, np.newaxis, np.newaxis]  # [C,H,W]
        # print(cam.shape)
        cam = np.sum(cam, axis=0)  # [H,W]
        cam = np.maximum(cam, 0)  # ReLU

        # 数值归一化
        cam -= np.min(cam)
        cam /= np.max(cam)
        # resize to 224*224
        # box = output[0]['instances'].pred_boxes.tensor[index].detach().numpy().astype(np.int32)
        box = res[0][0][index][:-1].detach().numpy().astype(np.int32)
        x1, y1, x2, y2 = box
        
        # cam = cv2.resize(cam, (x2 - x1, y2 - y1))
        # cam = cv2.resize(cam, (y2 - y1, x2 - x1)).T
        # print(cam.shape)

        # class_id = output[0]['instances'].pred_classes[index].detach().numpy()
        class_id = res[1][0][index].detach().numpy()
        plt.imshow(cam)
        return cam, box, class_id

In [21]:
grad_cam = GradCAM(model, 'backbone.layer4.2.conv1')

In [22]:
mask, box, class_id = grad_cam(data[0],0)

ModuleAttributeError: 'RoITransRoIHead' object has no attribute 'simple_test_bboxes'